In [1]:
import gta_prices
import numpy as np
import pandas as pd

In [2]:
def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
                and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'EMG'
periods = {'p1': 48, 'p2': 24, 'p3': 12, 'p4': 6}
position_size = 7500
risk_pct = 0.2
commission = 11.95
sduty = 0.5

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian lines, entries & exits for each look back period.
entry = None
for i, period in enumerate(periods.values()):
    i += 1
    globals()[f'p{i}'] = period
    df = pd.DataFrame(prices.copy())
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    df['sys'] = i
    df['period'] = period
    if i == 1:
        df['buy'] = np.where(df.close > df.upr, 1, 0)
    else:
        df['buy'] = entry
    df['sell'] = np.where(df.close < df.mid, 1, 0)
    df['state'] = state(df.buy, df.sell, period)
    df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
    if i == 1:
        entry = df.entry
    df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)
    globals()[f'dc{i}'] = df

In [6]:
# Trade list indexed by date.
td = pd.concat([dc1[dc1.entry == 1], dc1[dc1.exit == 1], dc2[dc2.exit == 1], dc3[dc3.exit == 1],
                dc4[dc4.exit == 1]], axis=0)
td = td.sort_index()

In [7]:
# Initial volatility adjusted position size on entry.
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

In [8]:
# Modify number of shares to be purchased to be divisible by 4.
s = pd.Series(td.shares.copy())
for i, share in enumerate(s):
    while s.iloc[i] % 4 != 0:
        s.iloc[i] += 1
td.shares = s

In [9]:
# Adjust risk amount on entry for revised share count.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0)

In [10]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [11]:
td

,open,high,low,close,upr,lwr,mid,sys,period,buy,sell,state,entry,exit,volatility,risk_amt,shares
date,,,,,,,,,,,,,,,,,
2001-05-18,1.6857,1.7695,1.6781,1.7676,1.7381,0.90857,1.323335,1,48,1,0,1,1,0,0.251338,5967.4176,3376
2001-07-13,1.8667,1.8743,1.7086,1.7219,1.9143,1.56000,1.737150,3,12,0,1,0,0,1,0.000000,0.0000,844
2001-07-13,1.8667,1.8743,1.7086,1.7219,1.9143,1.68760,1.800950,4,6,0,1,0,0,1,0.000000,0.0000,844
2001-07-20,1.7248,1.7248,1.5524,1.6000,1.9143,1.35900,1.636650,2,24,0,1,0,0,1,0.000000,0.0000,844
2002-02-22,2.2476,2.2476,1.8286,1.9048,2.6571,1.37520,2.016150,1,48,0,1,0,0,1,0.000000,0.0000,844
2003-05-16,2.0438,2.2400,2.0286,2.2286,2.1600,1.52860,1.844300,1,48,1,0,1,1,0,0.172440,8700.4544,3904
2003-07-25,2.4171,2.4286,2.3638,2.3771,2.5143,2.27050,2.392400,4,6,0,1,0,0,1,0.000000,0.0000,976
2003-08-22,2.3276,2.4057,2.2838,2.3410,2.5143,2.21520,2.364750,3,12,0,1,0,0,1,0.000000,0.0000,976
2004-05-14,3.1371,3.1695,3.0190,3.0971,3.5429,2.68190,3.112400,2,24,0,1,0,0,1,0.000000,0.0000,976


In [12]:
# Adjust calcs to handle any number of systems, ie. not tailored to 4.